In [14]:

import mysql.connector
from sqlalchemy import create_engine
import dask.dataframe as dd



# MySQL database connection settings
db_settings = {
    'host': 'localhost',
    'port':'3307',                   # Use the IP or hostname of your MySQL container
    'user': 'root',
    'password': '',
    'database': 'vinit',
}

In [15]:
base_mapping = {
    'B02512': 'Unter',
    'B02598': 'Hinter',
    'B02617': 'Weiter',
    'B02682': 'Schmecken',
    'B02764': 'Danach-NY',
    'B02765': 'Grun',
    'B02835': 'Dreist',
    'B02836': 'Drinnen'
}

file_paths = [
    "C://Users//jain vinit//Documents//FExam//uber-tlc-foil-response//uber-trip-data//uber-raw-data-aug14.csv", ]

In [16]:
ddf_list = [dd.read_csv(file) for file in file_paths]
df = dd.concat(ddf_list)

# Additional processing, mapping, etc.
df['Base'] = df['Base'].map(base_mapping.get, na_action='ignore')
print(df.head())

          Date/Time      Lat      Lon   Base
0  8/1/2014 0:03:00  40.7366 -73.9906  Unter
1  8/1/2014 0:09:00  40.7260 -73.9918  Unter
2  8/1/2014 0:12:00  40.7209 -74.0507  Unter
3  8/1/2014 0:12:00  40.7387 -73.9856  Unter
4  8/1/2014 0:12:00  40.7323 -74.0077  Unter


In [17]:
connection = mysql.connector.connect(**db_settings)
cursor = connection.cursor()

# Create the table if it doesn't exist
create_table_query = """
CREATE TABLE IF NOT EXISTS uber_data (
    Date DATE,
    Lat FLOAT,
    Lon FLOAT,
    Base VARCHAR(255)
);
"""
cursor.execute(create_table_query)
connection.commit()

In [18]:
#engine = create_engine(f"mysql+mysqlconnector://{db_settings['user']}:{db_settings['password']}@{db_settings['host']}/{db_settings['database']}")
#df.compute().to_sql('uber_data', con=engine, if_exists='replace', index=False)


# Update your connection string to use an empty password
engine = create_engine(f"mysql+mysqlconnector://{db_settings['user']}:{db_settings['password']}@{db_settings['host']}:{db_settings['port']}/{db_settings['database']}")


df.compute().to_sql('uber_data', con=engine, if_exists='replace', index=False)
# Close the MySQL connection
connection.close()
cursor.close()

True

In [19]:

import mysql.connector
from sqlalchemy import create_engine
import dask.dataframe as dd

# MySQL database connection settings
db_settings = {
    'host': 'localhost',
    'port': '3307',  # Use the IP or hostname of your MySQL container
    'user': 'root',
    'password': 'Wen2003',
    'database': 'vinit',
}

# Your existing code to read and process CSV files using Dask
base_mapping = {
    'B02512': 'Unter',
    'B02598': 'Hinter',
    'B02617': 'Weiter',
    'B02682': 'Schmecken',
    'B02764': 'Danach-NY',
    'B02765': 'Grun',
    'B02835': 'Dreist',
    'B02836': 'Drinnen'
}

file_paths = [
    "C://Users//jain vinit//Documents//FExam//uber-tlc-foil-response//uber-trip-data//uber-raw-data-aug14.csv",
    "C://Users//jain vinit//Documents//FExam//uber-tlc-foil-response//uber-trip-data//uber-raw-data-jul14.csv",

]

ddf_list = [dd.read_csv(file) for file in file_paths]
df = dd.concat(ddf_list)

# Additional processing, mapping, etc.
df['Base'] = df['Base'].map(base_mapping.get, na_action='ignore')

# MySQL connection using SQLAlchemy and connection pooling
engine = create_engine(
    f"mysql+mysqlconnector://{db_settings['user']}:{db_settings['password']}@{db_settings['host']}:{db_settings['port']}/{db_settings['database']}",
    pool_size=10,  # Adjust pool size based on your needs
    max_overflow=20
)

# Create the table if it doesn't exist
create_table_query = """
CREATE TABLE IF NOT EXISTS uber_data (
    Date DATE,
    Lat FLOAT,
    Lon FLOAT,
    Base VARCHAR(255)
);
"""
with engine.connect() as connection:
    connection.execute(create_table_query)

# Load CSV data into MySQL table
df.compute().to_sql('uber_data2', con=engine, if_exists='replace', index=False)


In [1]:
import mysql.connector
from sqlalchemy import create_engine
import dask.dataframe as dd
from dask.distributed import Client

# MySQL database connection settings
db_settings = {
    'host': 'localhost',
    'port': '3307',  # Use the IP or hostname of your MySQL container
    'user': 'root',
    'password': 'Wen2003',
    'database': 'vinit',
}

# Your existing code to read and process CSV files using Dask
base_mapping = {
    'B02512': 'Unter',
    'B02598': 'Hinter',
    'B02617': 'Weiter',
    'B02682': 'Schmecken',
    'B02764': 'Danach-NY',
    'B02765': 'Grun',
    'B02835': 'Dreist',
    'B02836': 'Drinnen'
}

file_paths = [
    "C://Users//jain vinit//Documents//FExam//uber-tlc-foil-response//uber-trip-data//uber-raw-data-aug14.csv",
    "C://Users//jain vinit//Documents//FExam//uber-tlc-foil-response//uber-trip-data//uber-raw-data-jul14.csv",
]

# Create a Dask distributed client to enable parallel processing
client = Client(dashboard_address=':8788')
client = Client()

# Read CSV files in parallel using Dask
ddf_list = [dd.read_csv(file, assume_missing=True) for file in file_paths]

# Additional processing, mapping, etc.
for i, ddf in enumerate(ddf_list):
    ddf_list[i] = ddf.map_partitions(lambda df: df.assign(Base=df['Base'].map(base_mapping)), meta=ddf)

# Concatenate Dask DataFrames
df = dd.concat(ddf_list)

# MySQL connection using SQLAlchemy and connection pooling
engine = create_engine(
    f"mysql+mysqlconnector://{db_settings['user']}:{db_settings['password']}@{db_settings['host']}:{db_settings['port']}/{db_settings['database']}",
    pool_size=10,  # Adjust pool size based on your needs
    max_overflow=20
)

# Create the table if it doesn't exist
create_table_query = """
CREATE TABLE IF NOT EXISTS uber_data (
    Date DATE,
    Lat FLOAT,
    Lon FLOAT,
    Base VARCHAR(255)
);
"""
with engine.connect() as connection:
    connection.execute(create_table_query)

# Load CSV data into MySQL table
df.compute().to_sql('uber_data', con=engine, if_exists='replace', index=False)

# Close the Dask distributed client
client.close()
